# EDA FRIS

In [136]:
#choose which dfs you want to import
number='0'

## Importing packages, data and functions

In [137]:
import pandas as pd
from ast import literal_eval
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [138]:
path = r'C:\Users\lucp11051\Documents\Jupyter_notebooks\0_Preparing_dataframes/'

publications_0 = pd.read_csv(path + number+'publications_0.csv')  
projects_0 = pd.read_csv(path + number+'projects_0.csv')  
persons_0 = pd.read_csv(path + number+'persons_0.csv')  
organisations_0 = pd.read_csv(path + number+'organisations_0.csv')  

In [139]:
path = r'C:\Users\lucp11051\Documents\Jupyter_notebooks\0_Preparing_dataframes/'

publications = pd.read_csv(path + number+'publications.csv')  
projects = pd.read_csv(path + number+'projects.csv')  
persons = pd.read_csv(path + number+'persons.csv')  
organisations = pd.read_csv(path + number+'organisations.csv')  

In [140]:
# function to bring a list of (level 4) disciplines to a list of level 2
def bring_to_level_2(list_of_disc):
    level_2_list=[]
    for disc in list_of_disc:
        level_2_list= level_2_list + [disc[:4]]
    return level_2_list
        

In [141]:
# function to get unique values 
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    # print list 
    return unique_list

In [142]:
# function that adds columns to the initial dataframe df_0:
# 'Level-2 disciplines '+ entity_string: the level 2 disciplines of a chosen entity
# 'Unique level-2 disciplines '+ entity_string: the unique level 2 disciplines of a chosen entity
# Example: (df_0, entity_df, entity_string) = (persons,organisations, 'Organisations')
def get_related_level2_disciplines(df_0, entity_df, entity_string):
    level2_col=[]
    for i in range(len(df_0)):
        level2=[]
        for entity_id in df_0[entity_string][i]:
            if len(entity_df[entity_df['ID'] == entity_id]['Level-2 disciplines']) == 0:
                entity_level2=[] 
            else:
                entity_level2=entity_df[entity_df['ID'] == entity_id]['Level-2 disciplines'].iloc[0]
            level2=level2 + [entity_level2]
        level2_col=level2_col+[level2]
    df_0['Level-2 disciplines '+ entity_string]=level2_col
#     df_0['Unique level-2 disciplines '+ entity_string]=df_0['Level-2 disciplines '+ entity_string].apply(lambda x: unique(x))

In [143]:
# function takes a list with (non-unique) codes and gives a discipline vector (np.array)
def make_discipline_vector(level2_list, discipline_codes):
    disc_vector = np.array([0]*len(discipline_codes))
    for i in range(len(discipline_codes)):
        disc_vector[i]=level2_list.count(discipline_codes[i])
        
    if sum(disc_vector)>0:
        normalized_vector=disc_vector/sum(disc_vector)
    else:
        normalized_vector=disc_vector
    return normalized_vector
        
    

In [144]:
def difference_in_months(datetime_end, datetime_start):
    time_difference = relativedelta(datetime_end, datetime_start)
    difference_in_years = time_difference.years
    if time_difference.days > 14:
        extra_month = 1
    else:
        extra_month = 0
    rest_months = time_difference.months +extra_month
    difference_in_months = difference_in_years*12 + rest_months
    
    return difference_in_months

In [145]:
def string_to_datetime(date_str):
    if 'Z' in date_str:
        date_str=date_str.replace('Z','')
    return datetime.strptime(date_str, '%Y-%m-%d').date()

In [146]:
def merge_aliases(df, entity):
    new_df=df.copy()
    for i in range(df.shape[0]):        
        all_ids = [df['ID'][i]] + df['Alias'][i]
        alias_df = df[df['ID'].isin(all_ids)]
        if entity == 'projects':
            new_df.loc[i,'Disciplines'] = str(alias_df['Disciplines'].sum())
            new_df.loc[i,'Persons'] = str(unique(alias_df['Persons'].sum()))
            new_df.loc[i,'Organisations'] = str(alias_df['Organisations'].sum())
            new_df.loc[i,'Publications'] = str(unique(alias_df['Publications'].sum()))
            new_df.loc[i,'Duration'] = alias_df['Duration'].mean()
        elif entity == 'persons':
            new_df.loc[i,'Disciplines'] = str(alias_df['Disciplines'].sum())
            new_df.loc[i,'Organisations'] = str(unique(alias_df['Organisations'].sum()))
            new_df.loc[i,'Publications'] = str(unique(alias_df['Publications'].sum()))
            new_df.loc[i,'Projects'] = str(unique(alias_df['Projects'].sum()))
        elif entity == 'publications':
            new_df.loc[i,'Disciplines'] = str(alias_df['Disciplines'].sum())
            new_df.loc[i,'Persons'] = str(unique(alias_df['Persons'].sum()))               
            new_df.loc[i,'Size'] = alias_df['Size'].mean()
            new_df.loc[i,'Disciplines WoS'] = str(alias_df['Disciplines WoS'].sum())
            new_df.loc[i,'Disciplines vabb'] = str(alias_df['Disciplines vabb'].sum())
            new_df.loc[i,'WoS ID'] = str(alias_df['WoS ID'].sum())
            new_df.loc[i,'vabb ID'] = str(alias_df['vabb ID'].sum())
        else:
            raise Exception('Entity should be either \'projects\', \'persons\' or \'publications\'.')
    return new_df

In [147]:
def remove_bookcomponent(list_of_pubs):
    clean_list=[]
    for pub_id in list_of_pubs:
        pub_id = pub_id.replace('book-component:', '')
        clean_list=clean_list+[pub_id]
    return clean_list

## Preparing dataframes

In [148]:
# delete invalid and duplicate projects

projects_total=projects.shape[0]
projects=projects.drop(projects[projects.Name == 'ERROR'].index)
projects_valid=projects.shape[0]
projects.drop_duplicates(subset=['ID'])
projects.index=range(projects_valid)

print(projects_valid/projects_total *100, 'percent of projects is valid.')

projects_0_total=projects_0.shape[0]
projects_0=projects_0.drop(projects_0[projects_0.Name == 'ERROR'].index)
projects_0_valid=projects_0.shape[0]
projects_0.drop_duplicates(subset=['ID'])
projects_0.index=range(projects_0_valid)

print(projects_0_valid/projects_0_total *100, 'percent of projects_0 is valid.')

99.91645781119465 percent of projects is valid.
100.0 percent of projects_0 is valid.


In [149]:
# merge aliases, add level2 disciplines, literal_eval, ...

projects['Disciplines'] = projects['Disciplines'].apply(lambda x: literal_eval(x))
projects['Persons'] = projects['Persons'].apply(lambda x: unique(literal_eval(x)))
projects['Organisations'] = projects['Organisations'].apply(lambda x: unique(literal_eval(x)))
projects['Publications'] = projects['Publications'].apply(lambda x: literal_eval(x))
projects['Alias'] = projects['Alias'].apply(lambda x: literal_eval(x))
projects['Start']=projects['Start'].apply(lambda x: string_to_datetime(x))
projects['End']=projects['End'].apply(lambda x: string_to_datetime(x))
length=[]
for i in range(projects.shape[0]):
    length=length + [difference_in_months(projects['End'][i],projects['Start'][i])]
projects['Duration']=length
projects = merge_aliases(projects, 'projects')

projects['Disciplines'] = projects['Disciplines'].apply(lambda x: literal_eval(x))
projects['Persons'] = projects['Persons'].apply(lambda x: unique(literal_eval(x)))
projects['Organisations'] = projects['Organisations'].apply(lambda x: unique(literal_eval(x)))
projects['Publications'] = projects['Publications'].apply(lambda x: unique(remove_bookcomponent(literal_eval(x))))
projects['Level-2 disciplines']=projects['Disciplines'].apply(lambda x: bring_to_level_2(x))
projects['Size']=projects['Persons'].apply(lambda x: len(x))

In [150]:
# merge aliases, add level2 disciplines, literal_eval, ...
projects_0['Disciplines'] = projects_0['Disciplines'].apply(lambda x: literal_eval(x))
projects_0['Persons'] = projects_0['Persons'].apply(lambda x: unique(literal_eval(x)))
projects_0['Organisations'] = projects_0['Organisations'].apply(lambda x: unique(literal_eval(x)))
projects_0['Publications'] = projects_0['Publications'].apply(lambda x: literal_eval(x))
projects_0['Alias'] = projects_0['Alias'].apply(lambda x: literal_eval(x))
projects_0['Start']=projects_0['Start'].apply(lambda x: string_to_datetime(x))
projects_0['End']=projects_0['End'].apply(lambda x: string_to_datetime(x))
length=[]
for i in range(projects_0.shape[0]):
    length=length + [difference_in_months(projects_0['End'][i],projects_0['Start'][i])]
projects_0['Duration']=length
projects_0 = merge_aliases(projects_0, 'projects')

projects_0['Disciplines'] = projects_0['Disciplines'].apply(lambda x: literal_eval(x))
projects_0['Persons'] = projects_0['Persons'].apply(lambda x: unique(literal_eval(x)))
projects_0['Organisations'] = projects_0['Organisations'].apply(lambda x: unique(literal_eval(x)))
projects_0['Publications'] = projects_0['Publications'].apply(lambda x: unique(remove_bookcomponent(literal_eval(x))))
projects_0['Level-2 disciplines']=projects_0['Disciplines'].apply(lambda x: bring_to_level_2(x))
projects_0['Size']=projects_0['Persons'].apply(lambda x: len(x))

In [151]:
# delete invalid and duplicate persons

persons_total=persons.shape[0]
persons=persons.drop(persons[persons.Name == 'ERROR'].index)
persons_valid=persons.shape[0]
persons.drop_duplicates(subset=['ID'])
persons.index=range(persons_valid)

print(persons_valid/persons_total *100, 'percent of persons is valid.')

persons_0_total=persons_0.shape[0]
persons_0=persons_0.drop(persons_0[persons_0.Name == 'ERROR'].index)
persons_0_valid=persons_0.shape[0]
persons_0.drop_duplicates(subset=['ID'])
persons_0.index=range(persons_0_valid)

print(persons_0_valid/persons_0_total *100, 'percent of persons_0 is valid.')

100.0 percent of persons is valid.
100.0 percent of persons_0 is valid.


In [152]:
# merge aliases, add level2 disciplines, literal_eval, ...
persons['Disciplines'] = persons['Disciplines'].apply(lambda x: literal_eval(x))
persons['Projects'] = persons['Projects'].apply(lambda x: literal_eval(x))
persons['Organisations'] = persons['Organisations'].apply(lambda x: literal_eval(x))
persons['Publications'] = persons['Publications'].apply(lambda x: literal_eval(x))
persons['Alias'] = persons['Alias'].apply(lambda x: literal_eval(x))
persons = merge_aliases(persons, 'persons')
persons['Disciplines'] = persons['Disciplines'].apply(lambda x: literal_eval(x))
persons['Projects'] = persons['Projects'].apply(lambda x: unique(literal_eval(x)))
persons['Organisations'] = persons['Organisations'].apply(lambda x: unique(literal_eval(x)))
persons['Publications'] = persons['Publications'].apply(lambda x: unique(remove_bookcomponent(literal_eval(x))))
persons['Level-2 disciplines']=persons['Disciplines'].apply(lambda x: bring_to_level_2(x))


In [153]:
# merge aliases, add level2 disciplines, literal_eval, ...
persons_0['Disciplines'] = persons_0['Disciplines'].apply(lambda x: literal_eval(x))
persons_0['Projects'] = persons_0['Projects'].apply(lambda x: literal_eval(x))
persons_0['Organisations'] = persons_0['Organisations'].apply(lambda x: literal_eval(x))
persons_0['Publications'] = persons_0['Publications'].apply(lambda x: literal_eval(x))
persons_0['Alias'] = persons_0['Alias'].apply(lambda x: literal_eval(x))
persons_0 = merge_aliases(persons_0, 'persons')
persons_0['Disciplines'] = persons_0['Disciplines'].apply(lambda x: literal_eval(x))
persons_0['Projects'] = persons_0['Projects'].apply(lambda x: unique(literal_eval(x)))
persons_0['Organisations'] = persons_0['Organisations'].apply(lambda x: unique(literal_eval(x)))
persons_0['Publications'] = persons_0['Publications'].apply(lambda x: unique(remove_bookcomponent(literal_eval(x))))
persons_0['Level-2 disciplines']=persons_0['Disciplines'].apply(lambda x: bring_to_level_2(x))


In [154]:
# delete invalid and duplicate publications

publications_total=publications.shape[0]
publications=publications.drop(publications[publications.Name == 'ERROR'].index)
publications_valid=publications.shape[0]
publications.drop_duplicates(subset=['ID'])
publications.index=range(publications_valid)

print(publications_valid/publications_total *100, 'percent of publications is valid.')

78.33859095688749 percent of publications is valid.


In [155]:
# merge aliases, add level2 disciplines, literal_eval, ...
publications['Size'] = publications['Size'].apply(lambda x: max(1,literal_eval(x)))
publications['WoS ID'] = publications['WoS ID'].apply(lambda x: literal_eval(x))
publications['vabb ID'] = publications['vabb ID'].apply(lambda x: literal_eval(x))
publications['Disciplines WoS'] = publications['Disciplines WoS'].apply(lambda x: literal_eval(x))
publications['Disciplines vabb'] = publications['Disciplines vabb'].apply(lambda x: literal_eval(x))
publications['Disciplines'] = publications['Disciplines'].apply(lambda x: literal_eval(x))
publications['Persons'] = publications['Persons'].apply(lambda x: unique(literal_eval(x)))
publications['Alias'] = publications['Alias'].apply(lambda x: literal_eval(x))
publications=merge_aliases(publications, 'publications')
publications['WoS ID'] = publications['WoS ID'].apply(lambda x: literal_eval(x))
publications['vabb ID'] = publications['vabb ID'].apply(lambda x: literal_eval(x))
publications['Disciplines WoS'] = publications['Disciplines WoS'].apply(lambda x: literal_eval(x))
publications['Disciplines vabb'] = publications['Disciplines vabb'].apply(lambda x: literal_eval(x))
publications['Disciplines'] = publications['Disciplines'].apply(lambda x: literal_eval(x))
publications['Persons'] = publications['Persons'].apply(lambda x: unique(literal_eval(x)))
publications['Level-2 disciplines']=publications['Disciplines'].apply(lambda x: bring_to_level_2(x))

In [156]:
# delete invalid and duplicate organisations

organisations_total=organisations.shape[0]
organisations=organisations.drop(organisations[organisations.Name == 'ERROR'].index)
organisations_valid=organisations.shape[0]
organisations.drop_duplicates(subset=['ID'])
organisations.index=range(organisations_valid)

print(organisations_valid/organisations_total *100, 'percent of organisations is valid.')

100.0 percent of organisations is valid.


In [157]:
organisations['Disciplines'] = organisations['Disciplines'].apply(lambda x: literal_eval(x))
organisations['Level-2 disciplines']=organisations['Disciplines'].apply(lambda x: bring_to_level_2(x))

## Projects

In [158]:
projects

,ID,Name,Start,End,Disciplines,Persons,Organisations,Publications,Alias,when_added,Duration,Level-2 disciplines,Size
0,fdde0090-f6ed-4938-91a6-13d370125cde,ICP Interuniversity Programme Physical land re...,2015-09-22,2016-09-30,"[04010303, 01070405, 01050313]",[b73e0e47-ad11-4ce7-a065-2993e6c3b602],"[2b317cdb-3a20-47d7-b2bd-75aa36eb9734, f28fd86...",[],[],0,12,"[0401, 0107, 0105]",1
1,0f80da13-5ac3-45c0-a0df-c2bd93051dbe,ERC Professorship GlycoTarget,2014-03-01,2019-02-28,"[010608, 030130, 030116, 030617, 030630, 03012...",[8834112c-33f1-4ee6-8a34-9c09a7b71535],[adbfa1d6-6a4b-4abf-8dc4-ea71c647af26],[],[],0,60,"[0106, 0301, 0301, 0306, 0306, 0301, 0106, 030...",1
2,0e185133-3ddf-4f8a-81d6-2ddc76a49cb6,EU Promotion of Democratic Governance via Func...,2016-07-01,2017-01-31,"[050604, 050602, 050699, 050607, 050606, 05060...","[f19619e2-340d-47ef-9792-35d2436febc4, e011bd3...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],[],[],0,7,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050...",2
3,fc7f8873-326f-41fe-9c75-08ea05d8058b,Somite development in 3D in vitro synthetic mi...,2018-01-01,2018-12-31,[010699],[05e00835-520a-4d3b-8e53-fbe03c1e43c7],[b9b3c61e-650e-4463-a0e2-6de8f49917ba],[],[],0,12,[0106],1
4,9be600ee-e306-4fac-8c4b-0e5134b9ff28,Housing for Refugee Inclusion exploring inclus...,2020-01-01,2023-12-31,"[02011199, 05040413, 05040702, 05060199, 06040...",[3de615bf-4167-4fa5-a4d3-0dac8248798a],[3420d399-29f4-4edf-9f9d-252e7b1d400d],[],[],0,48,"[0201, 0504, 0504, 0506, 0604]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,5177ba5f-64c4-4e9e-bf4e-d4eb5e4f25b9,Synthesis of benzodiocane ligin for applicatio...,2016-10-01,2019-09-30,"[020501, 020799, 01061007, 010606, 020601, 010...",[6c4f337a-6fac-41fc-9224-dc0ccbdeeebb],[37e63935-be43-4a88-8f83-97da12647a08],[],[],1,36,"[0205, 0207, 0106, 0106, 0206, 0106, 0302, 010...",1
1192,c641efba-1526-4018-80ba-2de9152d0e94,The soil systems under global change SOGLO,2012-10-01,2017-12-31,"[040103, 010704, 010503, 010604, 010701, 01070...","[bdf09bca-071b-4682-be43-6c47560ecab3, 47582bc...","[17cf03fe-441e-444b-87f4-62214f211431, f414cd4...",[],[fce55ffb-6026-455d-93c0-9389e0f52164],1,63,"[0401, 0107, 0105, 0106, 0107, 0107, 0107]",2
1193,6a2353c9-539f-47da-bc58-65f961a94d47,Congo basin integrated monitoring for forest c...,2010-12-15,2016-12-31,"[010601, 030505, 010401, 010606, 010610, 01060...","[9d445952-3146-4bce-8477-b3b681cf0b1f, bdf09bc...","[baf2328c-ac14-4990-8086-1126ac6cea11, 17cf03f...",[],[3ce1811f-5c7a-4854-8d8a-67e018819947],1,70,"[0106, 0305, 0104, 0106, 0106, 0106, 0107, 010...",3
1194,165ae5f2-aac4-4893-8597-240e351c60c7,ARBOREF integrated 39 lignin first 39 biorefinery,2015-04-01,2019-03-31,"[020602, 040102, 059999, 020404, 01061106, 020...","[a190c1ac-b82c-477b-8b16-39a5752d3e79, 6c4f337...","[b58d2d09-7716-4627-851a-d1608fec639f, 37e6393...",[],[c0c71b56-d67e-4445-870c-b6b7e9b0da77],1,48,"[0206, 0401, 0599, 0204, 0106, 0206, 0306, 020...",4


## projects_0

In [159]:
get_related_level2_disciplines(projects_0, persons, 'Persons')

get_related_level2_disciplines(projects_0, organisations, 'Organisations')

In [181]:
projects_0

,ID,Name,Start,End,Disciplines,Persons,Organisations,Publications,Alias,when_added,Duration,Level-2 disciplines,Size,Level-2 disciplines Persons,Level-2 disciplines Organisations
0,fdde0090-f6ed-4938-91a6-13d370125cde,ICP Interuniversity Programme Physical land re...,2015-09-22,2016-09-30,"[04010303, 01070405, 01050313]",[b73e0e47-ad11-4ce7-a065-2993e6c3b602],"[2b317cdb-3a20-47d7-b2bd-75aa36eb9734, f28fd86...",[],[],0,12,"[0401, 0107, 0105]",1,[[]],"[[0203, 0105, 0201], [0107, 0299, 0204, 0601, ..."
1,0f80da13-5ac3-45c0-a0df-c2bd93051dbe,ERC Professorship GlycoTarget,2014-03-01,2019-02-28,"[010608, 030130, 030116, 030617, 030630, 03012...",[8834112c-33f1-4ee6-8a34-9c09a7b71535],[adbfa1d6-6a4b-4abf-8dc4-ea71c647af26],[],[],0,60,"[0106, 0301, 0301, 0306, 0306, 0301, 0106, 030...",1,"[[0306, 0106, 0306, 0306, 0301, 0302, 0301, 03...","[[0106, 0106]]"
2,0e185133-3ddf-4f8a-81d6-2ddc76a49cb6,EU Promotion of Democratic Governance via Func...,2016-07-01,2017-01-31,"[050604, 050602, 050699, 050607, 050606, 05060...","[f19619e2-340d-47ef-9792-35d2436febc4, e011bd3...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],[],[],0,7,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050...",2,"[[0506, 0506, 0506], [0506, 0506, 0506, 0506, ...",[[0504]]
3,fc7f8873-326f-41fe-9c75-08ea05d8058b,Somite development in 3D in vitro synthetic mi...,2018-01-01,2018-12-31,[010699],[05e00835-520a-4d3b-8e53-fbe03c1e43c7],[b9b3c61e-650e-4463-a0e2-6de8f49917ba],[],[],0,12,[0106],1,[[0306]],"[[0301, 0306, 0301, 0306, 0301, 0304, 0306, 02..."
4,9be600ee-e306-4fac-8c4b-0e5134b9ff28,Housing for Refugee Inclusion exploring inclus...,2020-01-01,2023-12-31,"[02011199, 05040413, 05040702, 05060199, 06040...",[3de615bf-4167-4fa5-a4d3-0dac8248798a],[3420d399-29f4-4edf-9f9d-252e7b1d400d],[],[],0,48,"[0201, 0504, 0504, 0506, 0604]",1,"[[0201, 0201, 0604, 0604, 0604]]","[[0201, 0201]]"
5,ee843e8f-9c18-4c14-bfee-641e18fbd8e0,Lost tributaries of the Scheldt reconstructing...,2019-01-01,2019-12-31,"[06010119, 06010124, 06010108, 06010116]","[45230f30-5ba1-4ba1-a305-1307392c1cc1, acb40a5...",[eda251f4-4796-4ac5-a6de-0e58a70f58aa],[],[],0,12,"[0601, 0601, 0601, 0601]",2,"[[0601, 0105, 0105, 0601, 0601, 0601, 0601, 06...",[[0601]]
6,bf2e0a13-e0bf-48cd-8339-c8c256770b1c,The association between motor skill learning a...,2020-08-04,2024-08-04,"[03011804, 03011899, 03012306, 03012301, 03012...","[d86af692-304d-42dd-bb1f-86d0113be027, 24e9ba0...","[d4a49766-ed89-4de9-a06b-3c45f728354f, 3597372...",[],[],0,48,"[0301, 0301, 0301, 0301, 0301]",4,"[[0304], [0301, 0304, 0304, 0306], [0301, 0301...","[[0503], [0301, 0304, 0304, 0306]]"
7,7a8539e2-93ad-4864-a4c7-405ec848fdd4,Towards dynamic navigational charts for highly...,2020-09-04,2024-09-04,[02020301],"[e739bf7a-76c2-4e6e-8b6e-4dcf863be95f, de9b527...","[4ad02e7f-d5fa-4d66-98bb-d2f63929649a, 4c3d49c...",[],[],0,48,[0202],2,"[[0203], [0203]]","[[0203], [0203, 0203]]"
8,70295923-ab2c-44c6-ac9f-2ff4f28b5a75,Analysing the impact of head end tail end effe...,2020-05-14,2024-05-14,"[01070310, 04010302, 01050607]","[8c662e7b-3e22-4f37-bc2f-a4c7f8593181, 8d75503...","[f74d3c9c-2f54-4b95-9b41-61afcfe2a878, f1d3171...",[],[],0,48,"[0107, 0401, 0105]",3,"[[0105, 0106, 0107, 0107, 0201, 0202, 0299, 04...","[[0107], [0107]]"
9,c04fd85a-d90f-438d-b06a-e5606588ba84,2D materials optomechanics in Silicon photonics,2021-01-01,2021-06-30,"[01030999, 01030904]","[2e1cb018-8750-4c8e-a23b-a5244883a5a6, c405840...",[16a28d37-6005-49e4-be1b-46b8c76f6640],[],[],0,6,"[0103, 0103]",2,"[[0208, 0208, 0208, 0208], [0103, 0202, 0103, ...","[[0208, 0202, 0202]]"


In [161]:
# projects_0.to_csv('finished_' + number + 'projects_0.csv', index=False)

## Persons

In [162]:
persons

,ID,Name,Disciplines,Projects,Organisations,Publications,Alias,when_added,Level-2 disciplines
0,e011bd37-a5a1-46af-bf93-cf0a36b66ab5,Dmytro Panchuk,"[050699, 050603, 050604, 050601, 050602, 05060...",[0e185133-3ddf-4f8a-81d6-2ddc76a49cb6],"[086ed4b6-2916-4b9f-ac7c-05ad67f8c04a, 45ff2fc...","[7d514ae6-2050-400e-a9ba-6a6d4c0c1ae4, b342656...",[],0,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050..."
1,8834112c-33f1-4ee6-8a34-9c09a7b71535,Nico Callewaert,"[030604, 010609, 030617, 030616, 030116, 03020...","[15ec25c0-a04d-4209-bce7-63dc2d19405e, 74d24dc...","[0053b9e1-b5be-40f7-acc7-16b65e968727, c67d5ec...","[01ae477d-a0f4-4d69-9884-b108b89fdbb2, ad8304f...",[],0,"[0306, 0106, 0306, 0306, 0301, 0302, 0301, 030..."
2,b73e0e47-ad11-4ce7-a065-2993e6c3b602,Willy Bauwens,[],"[96a7ab46-5092-4601-ac3c-72292bffbd3f, 3872eff...","[f28fd86e-fce7-4c98-90fb-015b1a4e5523, d218893...","[be4ca9bf-e888-47ef-9187-d5111c8bcabc, 1b0b714...",[],0,[]
3,f19619e2-340d-47ef-9792-35d2436febc4,Jan Orbie,"[05060211, 05060204, 05060207]","[9299f1fc-e579-42c9-9ab0-9d8e98e37536, 9d874e7...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],"[59e05f27-58a5-474b-aea5-11983b6c7d83, afeed9e...",[],0,"[0506, 0506, 0506]"
4,45230f30-5ba1-4ba1-a305-1307392c1cc1,Jeroen Verhegge,"[06010124, 01050607, 01050606, 06010503, 06010...","[aaa9de7b-72a2-4110-b70c-7e6fe08915e6, ee843e8...","[41ec1094-0ea6-4da0-9777-3794a9a05440, eda251f...","[693c5c3b-429e-4231-ac95-8ee1e8822033, e79850a...",[],0,"[0601, 0105, 0105, 0601, 0601, 0601, 0601, 060..."
...,...,...,...,...,...,...,...,...,...
3769,fceb2b53-06c6-48a7-bad8-207e4c7df6d5,Herman Peiffer,"[020199, 020110]",[],[4bdf446c-b1de-4cb4-9872-07070924a4ee],"[1c2f77c9-1d61-4114-8e08-53cc526d0ecb, 45760ef...",[],1,"[0201, 0201]"
3770,a3855699-cf44-4e45-8ed6-d4c97b84c736,Jeroen De Maeyer,[02030104],[8cc324ad-6d0e-4f72-971b-9d0f4c7f454a],[5f3f66cc-4003-4831-9855-1a9db80e799b],[],[],1,[0203]
3771,1384722e-a9f8-469a-98f4-28b94091cd09,Ioan Mugurel Feraru,"[030303, 030602, 030301, 030202, 030102, 02080...",[],[37e63935-be43-4a88-8f83-97da12647a08],"[ce296dde-9ab7-4840-aa70-e3c57765f4ca, ac1f708...",[],1,"[0303, 0306, 0303, 0302, 0301, 0208, 0207, 020..."
3772,443c08c9-5928-493f-99d4-23d72b9ad0cf,Jo De Vrieze,"[01069999, 01060803, 02070299, 01060899, 02040...","[3ba09a79-3e6d-4dd9-9c84-bc37d9cc4dc1, 55dfba8...","[e4b3631f-abdd-45bf-a5e9-86c6f2e6e4d9, ce1ce9f...","[0a367678-55cf-4c06-bc75-96a5c83ff3c4, ba0ca1f...",[f1da3d6e-51f7-4a7a-b767-400bf91e016a],1,"[0106, 0106, 0207, 0106, 0204, 0204, 0207, 010..."


## Persons_0

In [163]:
# add total number of publication coauthors
persons_0['Coauthors Publications total count']=np.zeros(len(persons_0))
for i in range(len(persons_0)):
    coauthor_count=0
    for pub_id in persons_0['Publications'][i]:
        if len(publications[publications['ID'] == pub_id]['Size']) == 0:
            continue
        else:
            coauthor_count=coauthor_count+publications[publications['ID'] == pub_id]['Size'].iloc[0]-1
    persons_0.loc[i,'Coauthors Publications total count']=coauthor_count

In [164]:
persons_0

,ID,Name,Disciplines,Projects,Organisations,Publications,Alias,when_added,Level-2 disciplines,Coauthors Publications total count
0,e011bd37-a5a1-46af-bf93-cf0a36b66ab5,Dmytro Panchuk,"[050699, 050603, 050604, 050601, 050602, 05060...",[0e185133-3ddf-4f8a-81d6-2ddc76a49cb6],"[086ed4b6-2916-4b9f-ac7c-05ad67f8c04a, 45ff2fc...","[7d514ae6-2050-400e-a9ba-6a6d4c0c1ae4, b342656...",[],0,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050...",6.0
1,8834112c-33f1-4ee6-8a34-9c09a7b71535,Nico Callewaert,"[030604, 010609, 030617, 030616, 030116, 03020...","[15ec25c0-a04d-4209-bce7-63dc2d19405e, 74d24dc...","[0053b9e1-b5be-40f7-acc7-16b65e968727, c67d5ec...","[01ae477d-a0f4-4d69-9884-b108b89fdbb2, ad8304f...",[],0,"[0306, 0106, 0306, 0306, 0301, 0302, 0301, 030...",816.5
2,b73e0e47-ad11-4ce7-a065-2993e6c3b602,Willy Bauwens,[],"[96a7ab46-5092-4601-ac3c-72292bffbd3f, 3872eff...","[f28fd86e-fce7-4c98-90fb-015b1a4e5523, d218893...","[be4ca9bf-e888-47ef-9187-d5111c8bcabc, 1b0b714...",[],0,[],244.0
3,f19619e2-340d-47ef-9792-35d2436febc4,Jan Orbie,"[05060211, 05060204, 05060207]","[9299f1fc-e579-42c9-9ab0-9d8e98e37536, 9d874e7...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],"[59e05f27-58a5-474b-aea5-11983b6c7d83, afeed9e...",[],0,"[0506, 0506, 0506]",376.0
4,45230f30-5ba1-4ba1-a305-1307392c1cc1,Jeroen Verhegge,"[06010124, 01050607, 01050606, 06010503, 06010...","[aaa9de7b-72a2-4110-b70c-7e6fe08915e6, ee843e8...","[41ec1094-0ea6-4da0-9777-3794a9a05440, eda251f...","[693c5c3b-429e-4231-ac95-8ee1e8822033, e79850a...",[],0,"[0601, 0105, 0105, 0601, 0601, 0601, 0601, 060...",117.0
...,...,...,...,...,...,...,...,...,...,...
59,6c4f337a-6fac-41fc-9224-dc0ccbdeeebb,Wout Boerjan,[01061004],"[6c31ab62-2681-4e05-953a-2e439d440a8b, 1e306f8...",[37e63935-be43-4a88-8f83-97da12647a08],"[086fba17-4d80-4006-9f9d-52ed4d090e38, 860b4f9...",[],0,[0106],1144.0
60,4b74615b-d701-4448-a191-cb19436ee4cb,Dirk Vandepitte,[029999],"[017c8b7d-4efe-4647-97af-297bfd23595f, 4a7fc94...","[5f3f66cc-4003-4831-9855-1a9db80e799b, 22358ad...","[f9e8018c-49b3-4d66-9c12-4de3243c097b, e702163...",[],0,[0299],1186.0
61,b6fd60a4-c095-43ed-9743-67cd091db492,Lotte JANSSENS,"[03020301, 03022601, 03022602, 03040201, 03040...","[f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0, 062e2e1...","[172ee4c2-42bc-4e59-93f2-91f28a162f2a, eb6b8bd...","[59f0f484-5ad8-41d2-b115-613d61201ac7, eb7149a...",[64b0cfc6-e0a3-4efd-8576-480f849dec0d],0,"[0302, 0302, 0302, 0304, 0304, 0304, 0304, 030...",359.5
62,37b7164d-6bd4-470b-98d5-33b349dfd2b5,Nina Goossens,"[03040599, 030126, 030402, 030405, 030626]","[f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0, 062e2e1...","[07fce545-f735-4a6b-9b40-02814a257bca, b4b0de5...","[59f0f484-5ad8-41d2-b115-613d61201ac7, eb7149a...",[89a6c8c9-fe61-434e-b2be-402f07e8c751],0,"[0304, 0301, 0304, 0304, 0306]",157.5


In [165]:
# Adds the disciplines of 'Organisations', 'Projects', 'Publications' and Coauthors to persons_0,
# as well as the size and duration of 'Projects', 'Publications'.

def person_0_disciplines(entity_df, entity_string):
    level2_col=[]
    for i in range(len(persons_0)):
        level2=[]
        for entity_id in persons_0[entity_string][i]:
            if len(entity_df[entity_df['ID'] == entity_id]['Level-2 disciplines']) == 0:
                entity_level2=[]
                print('Cannot find '+ entity_string + ': ', entity_id)
            else:
                entity_level2=entity_df[entity_df['ID'] == entity_id]['Level-2 disciplines'].iloc[0]
            level2=level2 + [entity_level2]
        level2_col=level2_col+[level2]
    persons_0['Level-2 disciplines '+ entity_string]=level2_col

    if entity_string == 'Projects' or entity_string == 'Publications':
        size_duration_col=[]
        for i in range(len(persons_0)):
            size_duration_list=[]
            for entity_id in persons_0[entity_string][i]:
                if len(entity_df[entity_df['ID'] == entity_id]['Level-2 disciplines']) == 0:
                    proj_duration=0
                    proj_size=1000
                else:
                    proj_size = entity_df[entity_df['ID'] == entity_id]['Size'].iloc[0]
                    if entity_string == 'Projects':
                        proj_duration = entity_df[entity_df['ID'] == entity_id]['Duration'].iloc[0] 
                    else:
                        proj_duration = 1
                size_duration_list=size_duration_list+[(proj_size,proj_duration)]
            size_duration_col=size_duration_col+[size_duration_list]
        persons_0['Size and duration '+ entity_string]=size_duration_col

        coauthors_col=[]
        for i in range(len(persons_0)):
            per_id = persons_0['ID'][i]
            pers_proj_coauthors=[]
            for entity_id in persons_0[entity_string][i]:
                if len(entity_df[entity_df['ID'] == entity_id]['Level-2 disciplines']) == 0:
                    coauthor_ids = []
                else:
                    coauthor_ids = entity_df[entity_df['ID'] == entity_id]['Persons'].iloc[0].copy()
                    if per_id in coauthor_ids: coauthor_ids.remove(per_id)
                    else: print('ATTENTION: coauthors error: person {}'.format(per_id), entity_string + ': {}'.format(entity_id) )

                project_coauthors=[]
                for coauthor_id in coauthor_ids:
                    if len(persons[persons['ID'] == coauthor_id]['Level-2 disciplines']) == 0:
                        coauthor_disciplines = []
                        print('Cannot find coauthor ID: ', coauthor_id, 'from ' + entity_string, entity_id)
                    else:
                        coauthor_disciplines = persons[persons['ID'] == coauthor_id]['Level-2 disciplines'].iloc[0]

                    project_coauthors=project_coauthors+[coauthor_disciplines]
                pers_proj_coauthors=pers_proj_coauthors + [project_coauthors]
            coauthors_col=coauthors_col+[pers_proj_coauthors]
        persons_0['Coauthors '+ entity_string]=coauthors_col



person_0_disciplines(projects, 'Projects')
person_0_disciplines(organisations, 'Organisations')
person_0_disciplines(publications, 'Publications')

ATTENTION: coauthors error: person 89a6c8c9-fe61-434e-b2be-402f07e8c751 Projects: 3bc1f01e-9985-482a-a188-2e5bd1ed40c5
ATTENTION: coauthors error: person b6fd60a4-c095-43ed-9743-67cd091db492 Projects: e3bf8346-ec6f-48dc-95b4-41987f832d5f
ATTENTION: coauthors error: person b6fd60a4-c095-43ed-9743-67cd091db492 Projects: ba547409-fd3b-4566-9652-88997fec3c4f
ATTENTION: coauthors error: person b6fd60a4-c095-43ed-9743-67cd091db492 Projects: d958fd9a-cc28-447c-9c0b-88b76657507a
ATTENTION: coauthors error: person b6fd60a4-c095-43ed-9743-67cd091db492 Projects: 5dfd2fee-1fb8-47bf-92ab-5994c1f6a761
ATTENTION: coauthors error: person 37b7164d-6bd4-470b-98d5-33b349dfd2b5 Projects: f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0
ATTENTION: coauthors error: person 37b7164d-6bd4-470b-98d5-33b349dfd2b5 Projects: 062e2e14-f774-4b5e-8c8c-f4341943a113
ATTENTION: coauthors error: person 64b0cfc6-e0a3-4efd-8576-480f849dec0d Projects: f3da9a31-cf15-4e2e-bf14-4ce7773d2eb0
ATTENTION: coauthors error: person 64b0cfc6-e0a3

Cannot find Publications:  39413b8e-348a-4b3c-ac0e-5c10f518d75d
Cannot find Publications:  30ac60db-8d9c-42a5-b80b-65f716a34ff1
Cannot find Publications:  0b6edbf9-8c86-4429-bef5-89edff103cad
Cannot find Publications:  25085ee6-7c8a-47bb-855e-b68e6632f8ac
Cannot find Publications:  daa44d4e-bb08-4be0-847e-1001b0da792b
Cannot find Publications:  e3fbb302-442d-4db6-82ef-4bdf2a7ae63c
Cannot find Publications:  5264fe0c-594f-4de6-8bd1-eb2d48c2b811
Cannot find Publications:  4fefe7f5-029f-46a2-8bbc-c10c1700b53b
Cannot find Publications:  45f1d58c-8451-4c0c-99b4-9f6cdd3ffcdc
Cannot find Publications:  c0b8577f-88e6-4407-bc94-7615222e7e12
Cannot find Publications:  51fabe56-2e4e-4a55-a254-1e920c458843
Cannot find Publications:  611bdc11-731f-4928-9b58-5f14e36e9531
Cannot find Publications:  209843da-7db5-44b7-b403-5a0ab2713b09
Cannot find Publications:  afea61d9-d4e0-4784-a9ed-9841eac9399f
Cannot find Publications:  232e9283-4172-4e80-8c2a-eb988d32bef1
Cannot find Publications:  f4227667-c6bd

Cannot find Publications:  096de4c5-992a-460e-b5ff-f86a90f45f82
Cannot find Publications:  3cb5190f-278b-4b3f-a3ca-099d71c2c211
Cannot find Publications:  b762f9de-fb48-418f-a0a1-71ef76f42f04
Cannot find Publications:  5e8e7e71-e2d1-4e3e-939a-379b46799180
Cannot find Publications:  285f512f-eb84-4758-b3b6-b06ffc51a520
Cannot find Publications:  b726ff4a-03e5-42cd-9896-b21fef8caf06
Cannot find Publications:  75ae2475-a229-494c-97d4-cea675c3c2b7
Cannot find Publications:  744634fa-4c63-4bd5-b711-ab6c3702e9e6
Cannot find Publications:  024b0de7-0480-4224-a9a8-bf865619322e
Cannot find Publications:  14517c17-f032-47d2-aae6-033c06bc76c2
Cannot find Publications:  0413c666-2066-4b07-9b0a-ee0b7f4b85fd
Cannot find Publications:  c66ed2cb-1511-498d-972d-35ee5ed33cdf
Cannot find Publications:  4d2a0162-286a-469e-87d4-2b657cbe1943
Cannot find Publications:  b303004c-26f2-40f9-9564-c54cd3b3242e
Cannot find Publications:  4fb5df44-c403-4a87-a9d4-40315e079a64
Cannot find Publications:  5790fab2-a502

Cannot find Publications:  1ba6fc5c-27d0-44cf-8c20-ccc7d5ea50ae
Cannot find Publications:  dbd13af2-fc40-446e-b086-5f633ad42604
Cannot find Publications:  a763ad10-aa49-4f99-9203-dd048aa43de6
Cannot find Publications:  211f290f-564b-4cd3-9d6d-b202679465d8
Cannot find Publications:  299402fe-6488-4ae7-9238-a260393848f5
Cannot find Publications:  308f4706-ec88-4361-a354-04631f8906a7
Cannot find Publications:  30756f7c-ecde-4560-b6a9-a89ee82f958f
Cannot find Publications:  ad7a3d9a-5267-4c8f-aa8f-3dcfbff46f20
Cannot find Publications:  bab29182-844c-45e9-9719-3931d5222609
Cannot find Publications:  3399a7fb-e954-4187-93f8-aa76f47ab15b
Cannot find Publications:  d35c0bd9-a708-4057-b915-080dfa7efdaf
Cannot find Publications:  c746c2d0-f2fe-48e6-b389-1fb9b4d68ec8
Cannot find Publications:  9c4e8f29-4c38-4897-a0d5-94c70a3c1dae
Cannot find Publications:  7498b5a0-da03-415c-aa1e-7cacb4f8c935
Cannot find Publications:  540e4781-abba-446d-ad5d-95361d666d45
Cannot find Publications:  6e836753-97ae

Cannot find Publications:  23386c32-cbc8-4803-a936-3b9db034f6b3
Cannot find Publications:  0cf9dcb1-e9fb-48bf-9c67-5c2adfa96c19
Cannot find Publications:  48d7b78d-5895-413f-91aa-5e52f758bcb9
Cannot find Publications:  46c6b6c2-5d87-4353-af7c-11b934bcee00
Cannot find Publications:  aee13ce9-2333-4e03-af6e-b81458e025dd
Cannot find Publications:  423aa829-c365-4eb9-a7cc-fad209a21df3
Cannot find Publications:  bc60afcf-9e84-4240-b962-1d6cd5cd572b
Cannot find Publications:  f9a8b41a-1882-4dbf-ac7e-7cd66ae486f9
Cannot find Publications:  5e2a7840-2e74-4481-b20d-85190ae1aa2b
Cannot find Publications:  86c6d210-ed70-42ca-8ca4-e682a0779e1e
Cannot find Publications:  5be8ec82-3650-4233-a178-6c4bae417f33
Cannot find Publications:  324cd9a1-8375-4ff1-9783-1e5f768dc6c7
Cannot find Publications:  d612e5f8-5ce9-43b7-bcb9-0d038a8852bd
Cannot find Publications:  4629eb5b-dde5-48fe-8f12-fb2c3922f40f
Cannot find Publications:  4cfb61d4-7f4e-403a-bf6d-2a2d8eb674ed
Cannot find Publications:  10d9e10d-4185

...

Cannot find Publications:  2b1086ca-1ada-4ec2-bcc8-46114b6088c1
Cannot find Publications:  a4320ea5-3762-4e94-9c93-4ac25eaa66ef
Cannot find Publications:  65a38be8-ec16-4598-9287-e20f001fb5d6
Cannot find Publications:  6b41fb6f-bfbd-4b09-aee2-30cc46059773
Cannot find Publications:  5ab00158-e8cc-4353-87db-28c964fc13f5
Cannot find Publications:  78117df3-af1c-48da-807f-d9c04ce9cf48
Cannot find Publications:  f55bd388-439d-4833-bcb5-f8607102238f
Cannot find Publications:  8050e289-4043-422b-ac99-daa8d8447271
Cannot find Publications:  d38a38c6-8412-4237-a7e0-e7d3d53e1a3e
Cannot find Publications:  e0346b30-9929-4269-aad0-4ae1ead0e094
Cannot find Publications:  8f128f01-6da9-4db1-b761-24c1b9198919
Cannot find Publications:  23349a45-66eb-4630-ae45-d35b3c10fd9a
Cannot find Publications:  6c9cc620-ad1e-4477-8de9-77e976f7c7d5
Cannot find Publications:  3835b970-55df-4550-b9b5-8d2e3acad1fa
Cannot find Publications:  23e180b0-a6ed-4b8d-9139-634cc3924f20
Cannot find Publications:  e7c8b376-aead

Cannot find Publications:  04996a17-e7d4-413d-a315-ea8fb9aee93d
Cannot find Publications:  54d85280-4480-47a6-92fa-39480198d57d
Cannot find Publications:  de540c01-6497-4b26-b271-d978097ddc97
Cannot find Publications:  c903d9f8-c49f-41ac-8bd9-3a60e00db6da
Cannot find Publications:  0b4d33da-35de-497f-a46e-90c1632d251c
Cannot find Publications:  edc8c3bd-378f-4f7d-992e-0f74afe8f85e
Cannot find Publications:  08cfa7f8-223e-4425-aead-aa904598d28d
Cannot find Publications:  705f30fa-239c-460e-8f5b-97e5b784c7cf
Cannot find Publications:  e7db1d4c-f85b-4004-bf3f-ace25d96766c
Cannot find Publications:  74869819-1f9a-4f4d-be72-24dfe992b51a
Cannot find Publications:  c9cd08e0-05bf-471d-a9d9-22a1ac9b9765
Cannot find Publications:  320ff185-e8fc-419c-9dcc-0ba67710195e
Cannot find Publications:  fdd3cbc1-3d04-470b-ad67-3cdf1c3367cf
Cannot find Publications:  3f3256f9-2e98-4a5e-aa4d-77d688a90c42
Cannot find Publications:  891aea5f-b25f-4867-8c83-033eac7071d9
Cannot find Publications:  b908c0e7-cb70

Cannot find Publications:  dcacf0f5-58a0-4ab6-a706-1e1b3da0710e
Cannot find Publications:  41c7bab9-f32a-4c09-9871-8f6bcfcf1585
Cannot find Publications:  d483d8af-42fa-4747-9b8d-d5ff8d641ca0
Cannot find Publications:  7e01e23c-7ac8-46f6-a85d-4dc0f4f07ed8
Cannot find Publications:  e409ea72-e04e-46ed-ae02-2a6916ac1b39
Cannot find Publications:  6d0fce4c-bf9c-4d27-a58f-5bd0bf275a4e
Cannot find Publications:  e111cb16-d027-410b-af3c-b5373e8fe4ee
Cannot find Publications:  8f7ada09-5e40-49a2-a874-f7d5433f1876
Cannot find Publications:  eb2b0a87-0f02-4585-b441-45c2f2e97df5
Cannot find Publications:  ac70a1c6-67ba-4337-879f-c92bfe293315
Cannot find Publications:  d74da647-d6b2-4dcb-ab0c-3014f9a50901
Cannot find Publications:  ac5f14f7-5b18-4d83-bb3a-bc58f1e3691f
Cannot find Publications:  b4675ae7-537e-4933-a6e0-4704f1c11280
Cannot find Publications:  ae17559b-b264-41d9-847c-2ca274c76b12
Cannot find Publications:  0205c3b9-7f64-4878-b6a5-c159aebd377b
Cannot find Publications:  e8333ea9-e39b

Cannot find Publications:  c629497f-20ab-411b-b276-ac3baf39f6cd
Cannot find Publications:  5c952642-eea8-454e-99df-22a2d402a9c7
Cannot find Publications:  9e08e461-f532-4fe8-bcdc-163ae8e19675
Cannot find Publications:  a3541b13-8e8b-40f8-ae72-12f9f96ee983
Cannot find Publications:  fe4ceb24-c0b6-40a8-84b1-0a83b0ec3768
Cannot find Publications:  e2a0caab-2426-489f-98ab-76c0666ebbb3
Cannot find Publications:  de32c272-8ae3-4284-9672-907d6b4f61ba
Cannot find Publications:  2f351b1f-8c93-4311-9d67-66923381480d
Cannot find Publications:  913cbcfb-693f-4609-be99-51bc734c2b62
Cannot find Publications:  a65e7ecb-12d7-443c-8260-b113ed8cca3c
Cannot find Publications:  53e24c22-1d71-46d6-8332-3bfcba10053e
Cannot find Publications:  c400b4d0-f10f-4704-8442-202956908973
Cannot find Publications:  b97e778f-dc1c-41cd-a9d0-5c0f9f06c9ae
Cannot find Publications:  9c5c88f7-eb76-47f2-99aa-9a41bafbe545
Cannot find Publications:  cfcab61c-35f3-4b18-9345-cec031be8b7e
Cannot find Publications:  a182b494-b5d2

ATTENTION: coauthors error: person 89a6c8c9-fe61-434e-b2be-402f07e8c751 Publications: 45746922-01fb-4bd0-b193-478356124cf0
ATTENTION: coauthors error: person 89a6c8c9-fe61-434e-b2be-402f07e8c751 Publications: 2da56b90-e4a9-46d2-9bd3-3d2eac39b753
ATTENTION: coauthors error: person 89a6c8c9-fe61-434e-b2be-402f07e8c751 Publications: 306e38cb-8c0d-4eae-af8e-27d5b4449af3
ATTENTION: coauthors error: person 89a6c8c9-fe61-434e-b2be-402f07e8c751 Publications: 6d213b7f-5789-4b2d-a6d5-f4347236fa3c
ATTENTION: coauthors error: person 89a6c8c9-fe61-434e-b2be-402f07e8c751 Publications: bb96b6a8-f73c-49a5-bf65-e095bf274dc1
ATTENTION: coauthors error: person 89a6c8c9-fe61-434e-b2be-402f07e8c751 Publications: 4299953e-8c09-4776-a680-599f177d17f0
ATTENTION: coauthors error: person 89a6c8c9-fe61-434e-b2be-402f07e8c751 Publications: 24b80217-b8c9-454a-8173-a59ba1f09bd6
ATTENTION: coauthors error: person 89a6c8c9-fe61-434e-b2be-402f07e8c751 Publications: 74c345c4-babf-4849-a410-f9352157a95a
ATTENTION: coaut

In [166]:
persons_0

,ID,Name,Disciplines,Projects,Organisations,Publications,Alias,when_added,Level-2 disciplines,Coauthors Publications total count,Level-2 disciplines Projects,Size and duration Projects,Coauthors Projects,Level-2 disciplines Organisations,Level-2 disciplines Publications,Size and duration Publications,Coauthors Publications
0,e011bd37-a5a1-46af-bf93-cf0a36b66ab5,Dmytro Panchuk,"[050699, 050603, 050604, 050601, 050602, 05060...",[0e185133-3ddf-4f8a-81d6-2ddc76a49cb6],"[086ed4b6-2916-4b9f-ac7c-05ad67f8c04a, 45ff2fc...","[7d514ae6-2050-400e-a9ba-6a6d4c0c1ae4, b342656...",[],0,"[0506, 0506, 0506, 0506, 0506, 0506, 0506, 050...",6.0,"[[0506, 0506, 0506, 0506, 0506, 0506, 0506, 05...","[(2, 7)]","[[[0506, 0506, 0506]]]","[[0502], [0504]]","[[0506, 0507, 0506, 0504], [], [0506, 0506], [...","[(2.0, 1), (1000, 0), (3.0, 1), (1000, 0), (2....","[[[0506, 0506, 0506, 0506, 0506, 0506, 0506, 0..."
1,8834112c-33f1-4ee6-8a34-9c09a7b71535,Nico Callewaert,"[030604, 010609, 030617, 030616, 030116, 03020...","[15ec25c0-a04d-4209-bce7-63dc2d19405e, 74d24dc...","[0053b9e1-b5be-40f7-acc7-16b65e968727, c67d5ec...","[01ae477d-a0f4-4d69-9884-b108b89fdbb2, ad8304f...",[],0,"[0306, 0106, 0306, 0306, 0301, 0302, 0301, 030...",816.5,"[[0106, 0106, 0106], [0206, 0206, 0206, 0206],...","[(3, 24), (2, 24), (2, 24), (2, 24), (2, 12), ...","[[[0106, 0301, 0106, 0106, 0106, 0301], [0306,...","[[0104], [0301], [0106, 0106]]","[[], [], [], [], [], [], [], [], [], [], [], [...","[(11.0, 1), (1000, 0), (5.0, 1), (1000, 0), (5...","[[], [], [[0401, 0206, 0206, 0207, 0106]], [],..."
2,b73e0e47-ad11-4ce7-a065-2993e6c3b602,Willy Bauwens,[],"[96a7ab46-5092-4601-ac3c-72292bffbd3f, 3872eff...","[f28fd86e-fce7-4c98-90fb-015b1a4e5523, d218893...","[be4ca9bf-e888-47ef-9187-d5111c8bcabc, 1b0b714...",[],0,[],244.0,"[[0203, 0201, 0105], [0105, 0201], [0401, 0107...","[(2, 48), (2, 2), (1, 12), (1, 12), (1, 48), (...","[[[0201, 0201, 0201, 0201, 0201, 0201, 0201, 0...","[[0107, 0299, 0204, 0601, 0601, 0604, 0601, 02...","[[0207, 0299, 0303], [], [], [], [], [], [], [...","[(4.5, 1), (1000, 0), (3.0, 1), (1000, 0), (12...","[[[0107], [0107, 0208, 0107, 0107], [0107, 040..."
3,f19619e2-340d-47ef-9792-35d2436febc4,Jan Orbie,"[05060211, 05060204, 05060207]","[9299f1fc-e579-42c9-9ab0-9d8e98e37536, 9d874e7...",[45ff2fc2-f78b-489c-ae38-b6fca2022521],"[59e05f27-58a5-474b-aea5-11983b6c7d83, afeed9e...",[],0,"[0506, 0506, 0506]",376.0,"[[0502, 0506, 0506, 0506, 0506, 0506, 0506, 05...","[(2, 24), (2, 24), (1, 48), (12, 60), (1, 48),...","[[[0506, 0506, 0506, 0506, 0506, 0506, 0506, 0...",[[0504]],"[[], [], [], [], [], [], [], [], [], [0207], [...","[(2.0, 1), (1000, 0), (5.0, 1), (3.0, 1), (100...","[[[0506, 0506, 0506]], [], [[0506, 0506, 0506]..."
4,45230f30-5ba1-4ba1-a305-1307392c1cc1,Jeroen Verhegge,"[06010124, 01050607, 01050606, 06010503, 06010...","[aaa9de7b-72a2-4110-b70c-7e6fe08915e6, ee843e8...","[41ec1094-0ea6-4da0-9777-3794a9a05440, eda251f...","[693c5c3b-429e-4231-ac95-8ee1e8822033, e79850a...",[],0,"[0601, 0105, 0105, 0601, 0601, 0601, 0601, 060...",117.0,"[[0601, 0107, 0601, 0107, 0105], [0601, 0601, ...","[(4, 48), (2, 12), (2, 39)]","[[[0601, 0601, 0601, 0601, 0601, 0601, 0601, 0...","[[0107, 0107], [0601]]","[[], [], [], [], [], [], [], [], [], [], [], [...","[(2.0, 1), (1000, 0), (4.0, 1), (1000, 0), (7....","[[], [], [[0601, 0601, 0601, 0601, 0601, 0601,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,6c4f337a-6fac-41fc-9224-dc0ccbdeeebb,Wout Boerjan,[01061004],"[6c31ab62-2681-4e05-953a-2e439d440a8b, 1e306f8...",[37e63935-be43-4a88-8f83-97da12647a08],"[086fba17-4d80-4006-9f9d-52ed4d090e38, 860b4f9...",[],0,[0106],1144.0,"[[0106, 0106, 0106, 0106], [0207], [], [], [01...","[(3, 24), (2, 48), (3, 24), (2, 24), (2, 24), ...","[[[0106, 0106, 0106, 0106, 0301, 0106, 0106, 0...","[[0207, 0106, 0106]]","[[], [], [], [], [0306, 0106], [], [], [], [],...","[(12.0, 1), (1000, 0), (4.0, 1), (1000, 0), (7...","[[[0303, 0306, 0302, 0303, 0301, 0208, 0

In [167]:
# VODS disciplines codes:
discipline_codes=['0101',
'0102',
'0103',
'0104',
'0105',
'0106',
'0107',
'0199',
'0201',
'0202',
'0203',
'0204',
'0205',
'0206',
'0207',
'0208',
'0299',
'0301',
'0302',
'0303',
'0304',
'0305',
'0306',
'0399',
'0401',
'0402',
'0499',
'0501',
'0502',
'0503',
'0504',
'0505',
'0506',
'0507',
'0508',
'0599',
'0601',
'0602',
'0603',
'0604',
'0699',
'0700']


In [168]:
def add_vectors_with_weights(list_of_vectors_and_weights, size=42):
    total_vector=np.zeros(size)
    for (vector,weight) in list_of_vectors_and_weights:
        total_vector=total_vector + vector*weight
        
    if sum(total_vector)>0:
        normalized_vector=total_vector/sum(total_vector)
    else:
        normalized_vector=total_vector
    return normalized_vector
    
        

In [169]:
# From disciplines of organisations, projects or publications to the (weighted) aggregate discipline vector      

def disciplines_to_weighted_aggregate_vector(list_of_lists, discipline_codes, size_and_duration = None):
    list_of_vectors=[]
    for i in range(len(list_of_lists)):
        vector = make_discipline_vector(list_of_lists[i],discipline_codes)
        if size_and_duration is None:
            weight=1
        else:
            size = size_and_duration[i][0]
            duration = size_and_duration[i][1]
            weight = duration/size 
        list_of_vectors = list_of_vectors+[(vector,weight)]
    aggregate_vector = add_vectors_with_weights(list_of_vectors)
    return aggregate_vector

In [170]:
# counts the organisations, projects or publications with disciplines
# returns (weighted) fractions of organisations, projects or publications with disciplines and 
# total number of organisations, projects or publications

def weigted_non_empty_list_counter(list_of_lists, size_and_duration = None):
    if size_and_duration is not None:
        sum_of_weights = 0
        for (size,duration) in size_and_duration:
            sum_of_weights = sum_of_weights + duration/size
        
    
    empty_count=0
    for i in range(len(list_of_lists)):
        if size_and_duration is None:
            weight=1/len(list_of_lists)
        else:
            size = size_and_duration[i][0]
            duration = size_and_duration[i][1]
            weight = (duration/size)/sum_of_weights
        if list_of_lists[i] == []:
            empty_count=empty_count + weight   
    return min(len(list_of_lists),1-empty_count), len(list_of_lists)

In [171]:
# From disciplines of coauthors to the (weighted) aggregate discipline vector

def disciplines_to_weighted_aggregate_vector2(list_of_lists_of_lists, discipline_codes, size_and_duration = None):
    list_of_vectors=[]
    for i in range(len(list_of_lists_of_lists)):
        if len(list_of_lists_of_lists[i])== 0:
            continue
        if size_and_duration is None:
            weight=1
        else:
            size = size_and_duration[i][0]
            duration = size_and_duration[i][1]
            weight = duration/(size-1)
        for person_disc in list_of_lists_of_lists[i]:
            vector = make_discipline_vector(person_disc, discipline_codes)         
            list_of_vectors = list_of_vectors+[(vector,weight)]
            
    aggregate_vector = add_vectors_with_weights(list_of_vectors)
    return aggregate_vector

In [172]:
# counts the coauthors with disciplines
# returns (weighted) fractions of coauthors with disciplines and total number of coauthors

def weigted_non_empty_list_counter2(list_of_lists_of_lists, size_and_duration = None):
    number_of_coauthors=0
    for list_of_lists in list_of_lists_of_lists:
        number_of_coauthors=number_of_coauthors+len(list_of_lists)
    if number_of_coauthors==0:
        return 0,0
    
    if size_and_duration is not None:
        sum_of_weights = 0
        for (size,duration) in size_and_duration:
            if size==1:
                continue
            sum_of_weights = sum_of_weights + duration/(size-1)
    
    empty_count=0        
    for i in range(len(list_of_lists_of_lists)):
        if len(list_of_lists_of_lists[i]) == 0:
            continue
        if size_and_duration is None:
            weight=1/number_of_coauthors
        else:
            size = size_and_duration[i][0]
            duration = size_and_duration[i][1]
            weight = ((duration/(size-1))/sum_of_weights)/len(list_of_lists_of_lists[i])
        for person_disc in list_of_lists_of_lists[i]:
            if person_disc == []:
                empty_count=empty_count + weight
    return min(number_of_coauthors,1-empty_count), number_of_coauthors  

In [173]:
vectors_df= persons_0.copy()


vectors_df['Profile']=vectors_df['Level-2 disciplines'].apply(lambda x: disciplines_to_weighted_aggregate_vector([x],discipline_codes))
vectors_df['Profile count']=vectors_df['Level-2 disciplines'].apply(lambda x: weigted_non_empty_list_counter([x]))

entities=['Organisations', 'Projects', 'Publications']
for ent in entities:
    vectors_df[ent]=vectors_df['Level-2 disciplines '+ ent].apply(lambda x: disciplines_to_weighted_aggregate_vector(x,discipline_codes))
    vectors_df[ent+' count']=vectors_df['Level-2 disciplines '+ ent].apply(lambda x: weigted_non_empty_list_counter(x))
    if ent == 'Projects' or ent == 'Publications':
        vectors_df[ent + ' weighted'] = vectors_df.apply(lambda vectors_df:\
                disciplines_to_weighted_aggregate_vector(vectors_df['Level-2 disciplines '+ ent], discipline_codes, vectors_df['Size and duration '+ ent]), axis=1)
        vectors_df[ent + ' weighted count'] = vectors_df.apply(lambda vectors_df:\
                weigted_non_empty_list_counter(vectors_df['Level-2 disciplines '+ ent], vectors_df['Size and duration '+ ent]), axis=1)
        
        vectors_df['Coauthors '+ ent + ' weighted'] = vectors_df.apply(lambda vectors_df:\
                disciplines_to_weighted_aggregate_vector2(vectors_df['Coauthors '+ ent], discipline_codes, vectors_df['Size and duration '+ ent]), axis=1)
        vectors_df['Coauthors '+ ent + ' weighted count'] = vectors_df.apply(lambda vectors_df:\
                weigted_non_empty_list_counter2(vectors_df['Coauthors '+ ent], vectors_df['Size and duration '+ ent]), axis=1)
        
        vectors_df['Coauthors '+ ent + ' count'] = vectors_df.apply(lambda vectors_df:\
                weigted_non_empty_list_counter2(vectors_df['Coauthors '+ ent]), axis=1)
        vectors_df['Coauthors '+ ent] = vectors_df.apply(lambda vectors_df:\
                disciplines_to_weighted_aggregate_vector2(vectors_df['Coauthors '+ ent], discipline_codes), axis=1)

        
col=['ID','Profile','Profile count','Organisations', 'Organisations count', 'Projects', 'Projects count', 'Coauthors Projects', 'Coauthors Projects count',\
     'Publications', 'Publications count','Coauthors Publications', 'Coauthors Publications count', 'Coauthors Publications total count']
col_weighted=['ID','Profile','Profile count','Organisations', 'Organisations count', 'Projects weighted', 'Projects weighted count', 'Coauthors Projects weighted', 'Coauthors Projects weighted count',\
              'Publications weighted', 'Publications weighted count', 'Coauthors Publications weighted', 'Coauthors Publications weighted count',\
             'Coauthors Publications total count']

weighted_vectors_df= vectors_df[col_weighted]
vectors_df=vectors_df[col]

weighted_vectors_df.columns = col

for i in range(vectors_df.shape[0]):
    total = vectors_df.loc[i, 'Coauthors Publications total count']
    if total>0: 
        completeness=vectors_df.loc[i, 'Coauthors Publications count'][0]*vectors_df.loc[i, 'Coauthors Publications count'][1]/total
        vectors_df.loc[i, 'Coauthors Publications count'] = str((completeness,total))
vectors_df=vectors_df.drop(columns=['Coauthors Publications total count'])
for i in range(weighted_vectors_df.shape[0]):
    total = weighted_vectors_df.loc[i, 'Coauthors Publications total count']
    if total>0: 
        completeness=weighted_vectors_df.loc[i, 'Coauthors Publications count'][0]*weighted_vectors_df.loc[i, 'Coauthors Publications count'][1]/total
        weighted_vectors_df.loc[i, 'Coauthors Publications count'] = str((completeness,total))
weighted_vectors_df=weighted_vectors_df.drop(columns=['Coauthors Publications total count'])

In [174]:
weighted_vectors_df

,ID,Profile,Profile count,Organisations,Organisations count,Projects,Projects count,Coauthors Projects,Coauthors Projects count,Publications,Publications count,Coauthors Publications,Coauthors Publications count
0,e011bd37-a5a1-46af-bf93-cf0a36b66ab5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 2)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.5714285714285714, 8)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.8333333333333334, 6.0)"
1,8834112c-33f1-4ee6-8a34-9c09a7b71535,"[0.0, 0.0, 0.0, 0.05, 0.0, 0.25, 0.0, 0.0, 0.0...","(1, 1)","[0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.333...","(1, 3)","[0.0, 0.0008506555955416192, 0.0, 0.0010025583...","(0.9756549083255142, 49)","[0.027236519607843134, 0.003063725490196079, 0...","(1, 45)","[0.0, 0.0, 0.013861788761522172, 0.06912997488...","(0.6640359324826586, 176)","[0.0007688063050556794, 0.004796530055943835, ...","(0.4317550667068829, 816.5)"
2,b73e0e47-ad11-4ce7-a065-2993e6c3b602,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1)","[0.0, 0.0, 0.03125000000000001, 0.0, 0.3229166...","(0.6666666666666667, 3)","[0.019736842105263157, 0.0, 0.0, 0.01973684210...","(1, 7)","[0.0, 0.0, 0.005787037037037038, 0.00440917107...","(0.953125, 11)","[0.0, 0.0, 0.0, 0.0, 0.015154497146231056, 0.0...","(0.2577460787695556, 85)","[0.0018759033728223957, 0.02118171538828606, 0...","(0.6045274624670345, 244.0)"
3,f19619e2-340d-47ef-9792-35d2436febc4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.03447037701974865,...","(1, 30)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003...","(1, 49)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.5015773990631469, 201)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.4741607742427479, 376.0)"
4,45230f30-5ba1-4ba1-a305-1307392c1cc1,"[0.0, 0.0, 0.0, 0.0, 0.4166666666666667, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, ...","(1, 2)","[0.0, 0.0, 0.0, 0.0, 0.1734736842105263, 0.054...","(1, 3)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.08080808080808081,...","(1, 5)","[0.0, 0.0, 0.0, 0.0, 0.01691956492547335, 0.0,...","(0.37675806941212187, 46)","[0.04292612713005525, 0.0, 0.0, 0.0, 0.1669539...","(0.5470085470085471, 117.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,6c4f337a-6fac-41fc-9224-dc0ccbdeeebb,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, ...","(1, 1)","[0.0, 0.0, 0.0, 0.023130663144378068, 0.0, 0.4...","(0.9690603632313357, 24)","[0.000478433131494356, 0.03577158754658326, 0....","(1, 74)","[0.0, 0.0, 0.0, 0.030086997239808486, 0.0, 0.4...","(0.7711988468057345, 230)","[0.003248533530662017, 0.03442014217471854, 0....","(0.2939041107009262, 1144.0)"
60,4b74615b-d701-4448-a191-cb19436ee4cb,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 4)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.016...","(0.9593001893671744, 29)","[0.0, 0.0, 0.046201232032854214, 0.0, 0.0, 0.0...","(0.896887621762889, 49)","[0.0, 0.0, 0.17208839240804527, 0.199017419983...","(0.3743287424428139, 336)","[0.0004683225232716047, 0.001313510646078272, ...","(0.4192919542440675, 1186.0)"
61,b6fd60a4-c095-43ed-9743-67cd091db492,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.8333333333333334, 6)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 6)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 13)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.08316964285714283,...","(0.661632606255625, 95)","[0.0, 8.555960683950332e-05, 0.000171119213679...","(0.7628738180080209, 359.5)"
62,37b7164d-6bd4-470b-98d5-33b349dfd2b5,"[0.0, 0.0, 0.

In [175]:
vectors_df

,ID,Profile,Profile count,Organisations,Organisations count,Projects,Projects count,Coauthors Projects,Coauthors Projects count,Publications,Publications count,Coauthors Publications,Coauthors Publications count
0,e011bd37-a5a1-46af-bf93-cf0a36b66ab5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 2)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.375, 8)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.8333333333333334, 6.0)"
1,8834112c-33f1-4ee6-8a34-9c09a7b71535,"[0.0, 0.0, 0.0, 0.05, 0.0, 0.25, 0.0, 0.0, 0.0...","(1, 1)","[0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.333...","(1, 3)","[0.0, 0.0009671179883945844, 0.0, 0.0015197568...","(0.9591836734693877, 49)","[0.025925925925925932, 0.014814814814814817, 0...","(1, 45)","[0.0, 0.0, 0.019047619047619046, 0.06666666666...","(0.4034090909090904, 176)","[0.0008116883116883114, 0.005502189038494184, ...","(0.43110838946723823, 816.5)"
2,b73e0e47-ad11-4ce7-a065-2993e6c3b602,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1)","[0.0, 0.0, 0.03125000000000001, 0.0, 0.3229166...","(0.6666666666666667, 3)","[0.03571428571428572, 0.0, 0.0, 0.035714285714...","(1, 7)","[0.0, 0.0, 0.006944444444444444, 0.00529100529...","(0.8181818181818181, 11)","[0.0, 0.0, 0.0, 0.0, 0.018518518518518517, 0.0...","(0.21176470588235352, 85)","[0.0034482758620689655, 0.02413793103448276, 0...","(0.5942622950819673, 244.0)"
3,f19619e2-340d-47ef-9792-35d2436febc4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.016666666666666673...","(1, 30)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.017...","(1, 49)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.35323383084577276, 201)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.46808510638297873, 376.0)"
4,45230f30-5ba1-4ba1-a305-1307392c1cc1,"[0.0, 0.0, 0.0, 0.0, 0.4166666666666667, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, ...","(1, 2)","[0.0, 0.0, 0.0, 0.0, 0.1368421052631579, 0.035...","(1, 3)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1916666666666...","(1, 5)","[0.0, 0.0, 0.0, 0.0, 0.03076923076923077, 0.0,...","(0.2826086956521743, 46)","[0.03385416666666667, 0.0, 0.0, 0.0, 0.1606212...","(0.5470085470085471, 117.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,6c4f337a-6fac-41fc-9224-dc0ccbdeeebb,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, ...","(1, 1)","[0.0, 0.0, 0.0, 0.03576656837526403, 0.0, 0.46...","(0.9166666666666666, 24)","[0.004536004536004537, 0.05308192072897956, 0....","(1, 74)","[0.0, 0.0, 0.0, 0.036398467432950186, 0.0, 0.5...","(0.3826086956521759, 230)","[0.004401154401154403, 0.03527757264521963, 0....","(0.28846153846153844, 1144.0)"
60,4b74615b-d701-4448-a191-cb19436ee4cb,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 4)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.016...","(0.9655172413793104, 29)","[0.0, 0.0, 0.044444444444444446, 0.0, 0.0, 0.0...","(0.9183673469387755, 49)","[0.0, 0.0, 0.16909722222222223, 0.198263888888...","(0.2857142857142877, 336)","[0.0006493506493506494, 0.00101010101010101, 0...","(0.3709949409780782, 1186.0)"
61,b6fd60a4-c095-43ed-9743-67cd091db492,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 1)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.8333333333333334, 6)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 6)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1, 13)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.10984848484848482,...","(0.46315789473684243, 95)","[0.0, 0.00012391573729863696, 0.00024783147459...","(0.7482614742698191, 359.5)"
62,37b7164d-6bd4-470b-98d5-33b349dfd2b5,"[0.0, 0.0

In [176]:
weighted_vectors_df.to_csv(number + 'weighted_vectors_df.csv', index=False)
vectors_df.to_csv(number + 'vectors_df.csv', index=False)

## Publications

In [177]:
publications.head()


,ID,Name,Disciplines,WoS ID,Disciplines WoS,vabb ID,Disciplines vabb,Persons,Size,Alias,when_added,Level-2 disciplines
0,3d3661db-6fa2-49d5-9b62-59b0a7829b65,Wind-Structure Interaction Simulations of Oval...,[],[],[],[],[],"[5b652de3-41b8-4a25-81a1-e036dfb8530f, 0ba45b5...",4.0,[],0,[]
1,b3f0df09-9575-4e4c-a938-aa3fd9afdc27,Security and development in EU external relati...,[0506],[],[],[c:vabb:354510],[0506],"[87b376b3-73c2-4fdb-8d74-662d96a506e3, f19619e...",5.0,[],1,[0506]
2,63be7b38-79cb-4352-acae-a0c403928a75,Mechanical strain determines the site-specific...,[],"[000449269700003, 000449269700003]",[],[],[],"[9023ad63-f52c-4180-a946-511ce265c572, 375c304...",19.0,[6a2ebfd0-b8d0-4048-b089-ed7bb93467f1],1,[]
3,0db6ceae-fd5a-4962-ba67-35868707e8ca,Analysis of the drought recovery of Andosols o...,"[0207, 0299, 0207, 0299]","[000379419500018, 000379419500018]","[0207, 0299, 0207, 0299]",[],[],"[85e5549a-2e67-4b62-9833-c5c93116fb36, b73e0e4...",5.5,[e7cc377d-480e-437a-901a-7dd7239666d9],1,"[0207, 0299, 0207, 0299]"
4,b1556823-c194-4cac-86f9-c915bac4c73c,A new unconditionally stable and consistent qu...,"[0207, 0107, 0207, 0107]","[000405997000010, 000405997000010]","[0207, 0107, 0207, 0107]",[],[],"[6d44687b-ee9a-41da-8a2d-919609bb0d4d, 71b6839...",5.0,[0bd28ddb-c4d3-4ad7-8dac-b143c2c65878],1,"[0207, 0107, 0207, 0107]"


In [178]:
ids=['book-component:6cfeede2-83a4-4ffd-818d-1f645cbbe845', '6cfeede2-83a4-4ffd-818d-1f645cbbe845']

alias_df = publications[publications['ID'].isin(ids)]
alias_df

,ID,Name,Disciplines,WoS ID,Disciplines WoS,vabb ID,Disciplines vabb,Persons,Size,Alias,when_added,Level-2 disciplines
16,6cfeede2-83a4-4ffd-818d-1f645cbbe845,Engineering of yeast glycoprotein expression,[],[],[],[],[],"[6c278919-932d-4b36-b0d5-2142c0d42dd5, 8834112...",3.0,[],1,[]


In [179]:
alias_df['Size'].sum()

3.0

## Organisations

In [180]:
organisations.head()

,ID,Name,Disciplines,when_added,Level-2 disciplines
0,45ff2fc2-f78b-489c-ae38-b6fca2022521,Department of Political Sciences,[050404],0,[0504]
1,f28fd86e-fce7-4c98-90fb-015b1a4e5523,Earth System Sciences,"[010703, 029901, 020406, 060101, 060104, 06040...",0,"[0107, 0299, 0204, 0601, 0601, 0604, 0601, 020..."
2,2b317cdb-3a20-47d7-b2bd-75aa36eb9734,Hydrology and Hydraulic Engineering,"[020307, 010503, 020112]",0,"[0203, 0105, 0201]"
3,adbfa1d6-6a4b-4abf-8dc4-ea71c647af26,Department of Biochemistry and microbiology,"[010602, 010608]",0,"[0106, 0106]"
4,3420d399-29f4-4edf-9f9d-252e7b1d400d,"Urban Design, Urbanism, Landscape and Planning","[020109, 020111]",0,"[0201, 0201]"
